In [1]:
import numpy as np
import pandas as pd

In [5]:
df_bak = pd.read_excel('../01-Data/ordens_pre_process.xlsx')

In [6]:
df = df_bak.drop(['Unnamed: 0'], axis=1)
df.columns = df.columns.str.replace(' ', '_')

In [7]:
df = df[df['Inner'] == 1]
df = df[df['Loja'] == 6]
df = df[df['Classe_de_onda'] == 4]
df = df[df['Tipo_de_buffer'] == 'CX']

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 3935 to 31314
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Ordem_de_produção          51 non-null     int64         
 1   Data_da_ordem_de_produção  51 non-null     datetime64[ns]
 2   Produto                    51 non-null     int64         
 3   Comprimento_unit           51 non-null     int64         
 4   Largura_unit               51 non-null     int64         
 5   Altura_unit                51 non-null     int64         
 6   Peso_unit                  51 non-null     float64       
 7   Peças                      51 non-null     int64         
 8   Caixa_padrão               51 non-null     object        
 9   Loja                       51 non-null     int64         
 10  Classe_de_onda             51 non-null     int64         
 11  Tipo_de_buffer             51 non-null     object        
 12  Item_pai 

In [9]:
def first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit):
    # Sort items in decreasing order of volume
    df_sorted = df.sort_values(by='Volume_total', ascending=False)
    
    # Initialize bins list
    bins = []
    
    for index, item in df_sorted.iterrows():
        for bin in bins:
            if (bin['Volume'] + item['Volume_total']) <= bin_capacity_vol and \
                (bin['Peso_total'] + item['Peso_total']) <= bin_capacity_weight and \
                bin['Amt.Items'] <= bin_item_limit:
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']
                    break
        else:
            # Create new bin
            bins.append({'Items': [index], 'Volume': item['Volume_total'], 'Peso_total': item['Peso_total'], 'Amt.Items': item['Peças']})
    
    return bins

In [11]:
bin_capacity_vol = 83393894.40
bin_capacity_weight = 23
bin_item_limit = 170
packed_bins = first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit)
print("Packed bins:")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

Packed bins:
Bin 1: Amt. Items 20 (Total Volume: 83375243 cubic mm | Total Weight: 5.6400kg)
Bin 2: Amt. Items 24 (Total Volume: 83013681 cubic mm | Total Weight: 7.9740kg)
Bin 3: Amt. Items 21 (Total Volume: 83186828 cubic mm | Total Weight: 10.7000kg)
Bin 4: Amt. Items 28 (Total Volume: 82897543 cubic mm | Total Weight: 8.9790kg)
Bin 5: Amt. Items 7 (Total Volume: 13521665 cubic mm | Total Weight: 1.6150kg)
